In [3]:
# https://convokit.cornell.edu/documentation/politenessStrategies.html
!pip install convokit

  Using cached convokit-3.1.0.tar.gz (193 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached msgpack_numpy-0.4.8-py2.py3-none-any.whl.metadata (5.0 kB)
  Using cached spacy-3.8.6-cp312-cp312-macosx_11_0_arm64.whl.metadata (27 kB)
  Using cached scikit_learn-1.6.1-cp312-cp312-macosx_12_0_arm64.whl.metadata (31 kB)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached dill-0.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached joblib-1.5.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached clean_text-0.6.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached Unidecode-1.4.0-py3-none-any.whl.metadata (13 kB)
  Using cached pymongo-4.13.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (22 kB)
  Using cached dnspython-2.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached thinc-8.3.6-cp312-cp312-macosx_11_0_arm64.whl.metadata (15 kB)
  Using cached h5py-3.12.1-cp312-cp312-ma

In [11]:
import pandas as pd
from convokit import Corpus, Utterance, Speaker
from convokit import PolitenessStrategies

In [12]:
from convokit import TextParser

In [17]:
import spacy

In [8]:
df = pd.read_csv("merged_translated_transcripts.csv")
df

,filename,interaction_number,speaker,sentence_number,text,task_type,Initials,Birth month,Birth Year,Mother tongue,Sex,Date of experiment,Experimenter,Place experiment,Age at experiment,Condition,Language of experiment,Order of tasks,Appreciation,text_en
0,PAIK_PA_bot.docx,1,You,1,How do the Reasoning Smogs work?,Information,PA,11,2011,english / french,m,4/20/2025,Olga,Versailles,13.56,C,English,LC,3,How do the Reasoning Smogs work?
1,PAIK_PA_bot.docx,1,You,2,I'm done.,Information,PA,11,2011,english / french,m,4/20/2025,Olga,Versailles,13.56,C,English,LC,3,I'm done.
2,PAIK_PA_bot.docx,1,ChatGPT,1,The Reasoning Smogs work by using tricky state...,Information,PA,11,2011,english / french,m,4/20/2025,Olga,Versailles,13.56,C,English,LC,3,The Reasoning Smogs work by using tricky state...
3,PAIK_PA_bot.docx,1,ChatGPT,2,They’re meant to test if someone can figure ou...,Information,PA,11,2011,english / french,m,4/20/2025,Olga,Versailles,13.56,C,English,LC,3,They’re meant to test if someone can figure ou...
4,PAIK_PA_bot.docx,1,ChatGPT,3,Each Smog gives a reason that sounds okay at f...,Information,PA,11,2011,english / french,m,4/20/2025,Olga,Versailles,13.56,C,English,LC,3,Each Smog gives a reason that sounds okay at f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831,PAIK_AG_bot.docx,30,ChatGPT,6,Est-ce que ça t’intéresserait ?,Learning,AG,6,2011,french,m,4/20/2025,Olga,Versailles,13.98,T,French,LC,4,Would that interest you?
1832,PAIK_AG_bot.docx,31,You,1,"Oops, Sidney, where's your ChatGPT?",Learning,AG,6,2011,french,m,4/20/2025,Olga,Versailles,13.98,T,French,LC,4,"Oops, Sidney, where's your ChatGPT?"
1833,PAIK_AG_bot.docx,31,ChatGPT,1,"Ah, I see we switched to English now!",Learning,AG,6,2011,french,m,4/20/2025,Olga,Versailles,13.98,T,French,LC,4,"Ah, I see we switched to English now!"
1834,PAIK_AG_bot.docx,31,ChatGPT,2,"When you say “Sidney, where’s your ChatGPT?”—c...",Learning,AG,6,2011,french,m,4/20/2025,Olga,Versailles,13.98,T,French,LC,4,"When you say “Sidney, where’s your ChatGPT?”—c..."


In [14]:
corpus = Corpus(utterances=[
    Utterance(id=str(row.name), speaker=Speaker(id=row['speaker']), text=row['text_en'])
    for _, row in df.iterrows()
])
corpus

In [16]:
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 15.7 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [20]:
parser = TextParser()
corpus = parser.transform(corpus)


In [21]:
politeness = PolitenessStrategies()
corpus = politeness.transform(corpus, markers=True)
corpus

In [22]:
data = []
for utt in corpus.iter_utterances():
    data.append({
        'id': utt.id,
        'speaker': utt.speaker.id,
        'text': utt.text,
        'politeness_strategies': utt.meta.get('politeness_strategies', {}),
        'politeness_markers': utt.meta.get('politeness_markers', {})
    })
df_polite = pd.DataFrame(data)
df_polite

,id,speaker,text,politeness_strategies,politeness_markers
0,0,You,How do the Reasoning Smogs work?,"{'feature_politeness_==Please==': 0, 'feature_...","{'politeness_markers_==Please==': [], 'politen..."
1,1,You,I'm done.,"{'feature_politeness_==Please==': 0, 'feature_...","{'politeness_markers_==Please==': [], 'politen..."
2,2,ChatGPT,The Reasoning Smogs work by using tricky state...,"{'feature_politeness_==Please==': 0, 'feature_...","{'politeness_markers_==Please==': [], 'politen..."
3,3,ChatGPT,They’re meant to test if someone can figure ou...,"{'feature_politeness_==Please==': 0, 'feature_...","{'politeness_markers_==Please==': [], 'politen..."
4,4,ChatGPT,Each Smog gives a reason that sounds okay at f...,"{'feature_politeness_==Please==': 0, 'feature_...","{'politeness_markers_==Please==': [], 'politen..."
...,...,...,...,...,...
1831,1831,ChatGPT,Would that interest you?,"{'feature_politeness_==Please==': 0, 'feature_...","{'politeness_markers_==Please==': [], 'politen..."
1832,1832,You,"Oops, Sidney, where's your ChatGPT?","{'feature_politeness_==Please==': 0, 'feature_...","{'politeness_markers_==Please==': [], 'politen..."
1833,1833,ChatGPT,"Ah, I see we switched to English now!","{'feature_politeness_==Please==': 0, 'feature_...","{'politeness_markers_==Please==': [], 'politen..."
1834,1834,ChatGPT,"When you say “Sidney, where’s your ChatGPT?”—c...","{'feature_politeness_==Please==': 0, 'feature_...","{'politeness_markers_==Please==': [], 'politen..."


In [23]:
df_strategies = pd.json_normalize(df_polite['politeness_strategies'])
df_strategies

,feature_politeness_==Please==,feature_politeness_==Please_start==,feature_politeness_==HASHEDGE==,feature_politeness_==Indirect_(btw)==,feature_politeness_==Hedges==,feature_politeness_==Factuality==,feature_politeness_==Deference==,feature_politeness_==Gratitude==,feature_politeness_==Apologizing==,feature_politeness_==1st_person_pl.==,...,feature_politeness_==1st_person_start==,feature_politeness_==2nd_person==,feature_politeness_==2nd_person_start==,feature_politeness_==Indirect_(greeting)==,feature_politeness_==Direct_question==,feature_politeness_==Direct_start==,feature_politeness_==HASPOSITIVE==,feature_politeness_==HASNEGATIVE==,feature_politeness_==SUBJUNCTIVE==,feature_politeness_==INDICATIVE==
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1832,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
1833,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1834,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [24]:
df_markers = pd.json_normalize(df_polite['politeness_markers'])
df_markers

,politeness_markers_==Please==,politeness_markers_==Please_start==,politeness_markers_==HASHEDGE==,politeness_markers_==Indirect_(btw)==,politeness_markers_==Hedges==,politeness_markers_==Factuality==,politeness_markers_==Deference==,politeness_markers_==Gratitude==,politeness_markers_==Apologizing==,politeness_markers_==1st_person_pl.==,...,politeness_markers_==1st_person_start==,politeness_markers_==2nd_person==,politeness_markers_==2nd_person_start==,politeness_markers_==Indirect_(greeting)==,politeness_markers_==Direct_question==,politeness_markers_==Direct_start==,politeness_markers_==HASPOSITIVE==,politeness_markers_==HASNEGATIVE==,politeness_markers_==SUBJUNCTIVE==,politeness_markers_==INDICATIVE==
0,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],"[[(how, 0, 0)]]",[],"[[(work, 0, 5)]]",[],[],[]
1,[],[],[],[],[],[],[],[],[],[],...,"[[(i, 0, 0)]]",[],[],[],[],[],[],[],[],[]
2,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],"[[(work, 0, 3)]]","[[(tricky, 0, 6)], [(confuse, 0, 9)], [(distra...",[],[]
3,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
4,[],[],[],[],[],"[[(really, 0, 20)]]",[],[],[],[],...,[],"[[(you, 0, 13)]]",[],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831,[],[],[],[],[],[],[],[],[],[],...,[],"[[(you, 0, 3)]]",[],[],[],[],[],[],[],[]
1832,[],[],[],[],[],[],[],[],"[[(oops, 0, 0)]]",[],...,[],"[[(your, 0, 6)]]",[],[],[],[],[],[],[],[]
1833,[],[],[],[],[],[],[],[],[],"[[(we, 0, 4)]]",...,[],[],[],[],[],[],[],[],[],[]
1834,[],[],[],[],[],[],[],[],[],[],...,[],"[[(you, 0, 1)], [(your, 0, 8)], [(you, 0, 10)]...",[],[],[],[],[],[],[],[]


In [25]:
# Add prefixes for clarity
df_strategies.columns = [f"strategy::{col}" for col in df_strategies.columns]
df_markers.columns = [f"marker::{col}" for col in df_markers.columns]

# Combine all data into a single DataFrame
df_combined = pd.concat([df_polite.drop(['politeness_strategies', 'politeness_markers'], axis=1),
                         df_strategies, df_markers], axis=1)


In [26]:
df_combined

,id,speaker,text,strategy::feature_politeness_==Please==,strategy::feature_politeness_==Please_start==,strategy::feature_politeness_==HASHEDGE==,strategy::feature_politeness_==Indirect_(btw)==,strategy::feature_politeness_==Hedges==,strategy::feature_politeness_==Factuality==,strategy::feature_politeness_==Deference==,...,marker::politeness_markers_==1st_person_start==,marker::politeness_markers_==2nd_person==,marker::politeness_markers_==2nd_person_start==,marker::politeness_markers_==Indirect_(greeting)==,marker::politeness_markers_==Direct_question==,marker::politeness_markers_==Direct_start==,marker::politeness_markers_==HASPOSITIVE==,marker::politeness_markers_==HASNEGATIVE==,marker::politeness_markers_==SUBJUNCTIVE==,marker::politeness_markers_==INDICATIVE==
0,0,You,How do the Reasoning Smogs work?,0,0,0,0,0,0,0,...,[],[],[],[],"[[(how, 0, 0)]]",[],"[[(work, 0, 5)]]",[],[],[]
1,1,You,I'm done.,0,0,0,0,0,0,0,...,"[[(i, 0, 0)]]",[],[],[],[],[],[],[],[],[]
2,2,ChatGPT,The Reasoning Smogs work by using tricky state...,0,0,0,0,0,0,0,...,[],[],[],[],[],[],"[[(work, 0, 3)]]","[[(tricky, 0, 6)], [(confuse, 0, 9)], [(distra...",[],[]
3,3,ChatGPT,They’re meant to test if someone can figure ou...,0,0,0,0,0,0,0,...,[],[],[],[],[],[],[],[],[],[]
4,4,ChatGPT,Each Smog gives a reason that sounds okay at f...,0,0,0,0,0,1,0,...,[],"[[(you, 0, 13)]]",[],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831,1831,ChatGPT,Would that interest you?,0,0,0,0,0,0,0,...,[],"[[(you, 0, 3)]]",[],[],[],[],[],[],[],[]
1832,1832,You,"Oops, Sidney, where's your ChatGPT?",0,0,0,0,0,0,0,...,[],"[[(your, 0, 6)]]",[],[],[],[],[],[],[],[]
1833,1833,ChatGPT,"Ah, I see we switched to English now!",0,0,0,0,0,0,0,...,[],[],[],[],[],[],[],[],[],[]
1834,1834,ChatGPT,"When you say “Sidney, where’s your ChatGPT?”—c...",0,0,0,0,0,0,0,...,[],"[[(you, 0, 1)], [(your, 0, 8)], [(you, 0, 10)]...",[],[],[],[],[],[],[],[]


In [29]:
# Concatenate results back to original DataFrame
df_combined_full = pd.concat([df, df_combined], axis=1)
df_combined_full

,filename,interaction_number,speaker,sentence_number,text,task_type,Initials,Birth month,Birth Year,Mother tongue,...,marker::politeness_markers_==1st_person_start==,marker::politeness_markers_==2nd_person==,marker::politeness_markers_==2nd_person_start==,marker::politeness_markers_==Indirect_(greeting)==,marker::politeness_markers_==Direct_question==,marker::politeness_markers_==Direct_start==,marker::politeness_markers_==HASPOSITIVE==,marker::politeness_markers_==HASNEGATIVE==,marker::politeness_markers_==SUBJUNCTIVE==,marker::politeness_markers_==INDICATIVE==
0,PAIK_PA_bot.docx,1,You,1,How do the Reasoning Smogs work?,Information,PA,11,2011,english / french,...,[],[],[],[],"[[(how, 0, 0)]]",[],"[[(work, 0, 5)]]",[],[],[]
1,PAIK_PA_bot.docx,1,You,2,I'm done.,Information,PA,11,2011,english / french,...,"[[(i, 0, 0)]]",[],[],[],[],[],[],[],[],[]
2,PAIK_PA_bot.docx,1,ChatGPT,1,The Reasoning Smogs work by using tricky state...,Information,PA,11,2011,english / french,...,[],[],[],[],[],[],"[[(work, 0, 3)]]","[[(tricky, 0, 6)], [(confuse, 0, 9)], [(distra...",[],[]
3,PAIK_PA_bot.docx,1,ChatGPT,2,They’re meant to test if someone can figure ou...,Information,PA,11,2011,english / french,...,[],[],[],[],[],[],[],[],[],[]
4,PAIK_PA_bot.docx,1,ChatGPT,3,Each Smog gives a reason that sounds okay at f...,Information,PA,11,2011,english / french,...,[],"[[(you, 0, 13)]]",[],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831,PAIK_AG_bot.docx,30,ChatGPT,6,Est-ce que ça t’intéresserait ?,Learning,AG,6,2011,french,...,[],"[[(you, 0, 3)]]",[],[],[],[],[],[],[],[]
1832,PAIK_AG_bot.docx,31,You,1,"Oops, Sidney, where's your ChatGPT?",Learning,AG,6,2011,french,...,[],"[[(your, 0, 6)]]",[],[],[],[],[],[],[],[]
1833,PAIK_AG_bot.docx,31,ChatGPT,1,"Ah, I see we switched to English now!",Learning,AG,6,2011,french,...,[],[],[],[],[],[],[],[],[],[]
1834,PAIK_AG_bot.docx,31,ChatGPT,2,"When you say “Sidney, where’s your ChatGPT?”—c...",Learning,AG,6,2011,french,...,[],"[[(you, 0, 1)], [(your, 0, 8)], [(you, 0, 10)]...",[],[],[],[],[],[],[],[]


In [27]:
output_politeness_strategies = "/Users/olga/Olga's workspace/Politeness Study/politeness_strategy.csv"

In [30]:
# Save
df_combined_full.to_csv(output_politeness_strategies, index=False)